In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
data = pd.read_csv("archive/all_mtg_cards.csv",low_memory=False)
data.head()

,name,multiverse_id,layout,names,mana_cost,cmc,colors,color_identity,type,supertypes,...,foreign_names,printings,original_text,original_type,legalities,source,image_url,set,set_name,id
0,Ancestor's Chosen,130550.0,normal,NaN,{5}{W}{W},7.0,['W'],['W'],Creature — Human Cleric,NaN,...,"[{'name': 'Ausgewählter der Ahnfrau', 'text': ...","['10E', 'JUD', 'UMA']",First strike\nWhen Ancestor's Chosen comes int...,Creature - Human Cleric,"[{'format': 'Commander', 'legality': 'Legal'},...",NaN,http://gatherer.wizards.com/Handlers/Image.ash...,10E,Tenth Edition,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c
1,Ancestor's Chosen,NaN,normal,NaN,{5}{W}{W},7.0,['W'],['W'],Creature — Human Cleric,NaN,...,NaN,"['10E', 'JUD', 'UMA']",NaN,NaN,"[{'format': 'Commander', 'legality': 'Legal'},...",NaN,NaN,10E,Tenth Edition,b7c19924-b4bf-56fc-aa73-f586e940bd42
2,Angel of Mercy,129465.0,normal,NaN,{4}{W},5.0,['W'],['W'],Creature — Angel,NaN,...,"[{'name': 'Engel der Gnade', 'text': 'Fliegend...","['10E', '8ED', '9ED', 'DDC', 'DVD', 'IMA', 'IN...","Flying\nWhen Angel of Mercy comes into play, y...",Creature - Angel,"[{'format': 'Brawl', 'legality': 'Legal'}, {'f...",NaN,http://gatherer.wizards.com/Handlers/Image.ash...,10E,Tenth Edition,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c
3,Angel of Mercy,NaN,normal,NaN,{4}{W},5.0,['W'],['W'],Creature — Angel,NaN,...,NaN,"['10E', '8ED', '9ED', 'DDC', 'DVD', 'IMA', 'IN...",NaN,NaN,"[{'format': 'Brawl', 'legality': 'Legal'}, {'f...",NaN,NaN,10E,Tenth Edition,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a
4,Angelic Blessing,129711.0,normal,NaN,{2}{W},3.0,['W'],['W'],Sorcery,NaN,...,"[{'name': 'Himmlischer Segen', 'text': 'Eine K...","['10E', '9ED', 'EXO', 'P02', 'POR', 'PS11', 'S...",Target creature gets +3/+3 and gains flying un...,Sorcery,"[{'format': 'Commander', 'legality': 'Legal'},...",NaN,http://gatherer.wizards.com/Handlers/Image.ash...,10E,Tenth Edition,55bd38ca-dc73-5c06-8f80-a6ddd2f44382


In [3]:
data.shape

(93643, 39)

## Analisa das colunas

In [4]:
na = data.isna().sum()
na.sort_values()

name                  0
layout                0
cmc                   0
type                  0
rarity                0
number                0
printings             0
set_name              0
id                    0
set                   0
artist                9
text               1095
legalities         4813
color_identity    10471
mana_cost         12378
colors            20408
multiverse_id     29835
image_url         29835
subtypes          36024
original_type     37499
rulings           38289
original_text     38643
foreign_names     39793
flavor            46891
power             49191
toughness         49191
variations        65155
supertypes        77065
watermark         86978
loyalty           92131
hand              93524
life              93524
release_date      93643
reserved          93643
starter           93643
timeshifted       93643
border            93643
names             93643
source            93643
dtype: int64

In [5]:
drop_na = na[na==data.shape[0]].index
drop_na

Index(['names', 'border', 'timeshifted', 'reserved', 'release_date', 'starter',
       'source'],
      dtype='object')

In [6]:
data.columns

Index(['name', 'multiverse_id', 'layout', 'names', 'mana_cost', 'cmc',
       'colors', 'color_identity', 'type', 'supertypes', 'subtypes', 'rarity',
       'text', 'flavor', 'artist', 'number', 'power', 'toughness', 'loyalty',
       'variations', 'watermark', 'border', 'timeshifted', 'hand', 'life',
       'reserved', 'release_date', 'starter', 'rulings', 'foreign_names',
       'printings', 'original_text', 'original_type', 'legalities', 'source',
       'image_url', 'set', 'set_name', 'id'],
      dtype='object')

In [7]:
data.drop(drop_na,axis=1,inplace=True)
data.columns


Index(['name', 'multiverse_id', 'layout', 'mana_cost', 'cmc', 'colors',
       'color_identity', 'type', 'supertypes', 'subtypes', 'rarity', 'text',
       'flavor', 'artist', 'number', 'power', 'toughness', 'loyalty',
       'variations', 'watermark', 'hand', 'life', 'rulings', 'foreign_names',
       'printings', 'original_text', 'original_type', 'legalities',
       'image_url', 'set', 'set_name', 'id'],
      dtype='object')

In [8]:
data[~data['set_name'].isna()][['name','set_name']]
# data[['name','life']]

,name,set_name
0,Ancestor's Chosen,Tenth Edition
1,Ancestor's Chosen,Tenth Edition
2,Angel of Mercy,Tenth Edition
3,Angel of Mercy,Tenth Edition
4,Angelic Blessing,Tenth Edition
...,...,...
93638,Into the Roil,Zendikar Rising
93639,Bloodchief's Thirst,Zendikar Rising
93640,Roil Eruption,Zendikar Rising
93641,Roiling Regrowth,Zendikar Rising


In [8]:
data_fi = data[['name','layout','mana_cost','cmc','colors','color_identity','type','supertypes',
                'subtypes', 'rarity', 'artist','power','toughness','loyalty','printings','original_text','set_name','set']]

#### layout

In [9]:
data_fi['layout'].isna().sum()

np.int64(0)

In [10]:
data_fi['layout'].unique()

array(['normal', 'saga', 'adventure', 'class', 'aftermath', 'split',
       'flip', 'transform', 'prototype', 'meld', 'leveler', 'mutate',
       'vanguard', 'modal_dfc', 'case', 'planar', 'scheme',
       'reversible_card', 'augment', 'host'], dtype=object)

#### mana_cost

In [11]:
data_fi['mana_cost']

0        {5}{W}{W}
1        {5}{W}{W}
2           {4}{W}
3           {4}{W}
4           {2}{W}
           ...    
93638       {1}{U}
93639          {B}
93640       {1}{R}
93641       {2}{G}
93642    {1}{R}{W}
Name: mana_cost, Length: 93643, dtype: object

In [12]:

data_fi.loc[:,'mana_cost'] = data_fi['mana_cost'].str.replace(r"[{}]", "", regex=True)

In [13]:
data_fi['mana_cost']

0        5WW
1        5WW
2         4W
3         4W
4         2W
        ... 
93638     1U
93639      B
93640     1R
93641     2G
93642    1RW
Name: mana_cost, Length: 93643, dtype: object

#### CMD

In [14]:
data_fi['cmc'].isna().sum()

np.int64(0)

In [15]:
data_fi['cmc']

0        7.0
1        7.0
2        5.0
3        5.0
4        3.0
        ... 
93638    2.0
93639    1.0
93640    2.0
93641    3.0
93642    3.0
Name: cmc, Length: 93643, dtype: float64

#### colors

In [16]:
data_fi['colors'].isna().sum()

np.int64(20408)

In [17]:
data_fi.loc[data_fi['colors'].isna(),'colors'] = 'C'

In [18]:
data_fi[data_fi['colors'].isna()]['supertypes'].unique()

array([], dtype=object)

In [19]:
data_fi['colors']

data_fi.loc[:, 'colors'] = data_fi['colors'].apply(lambda x:re.sub(r'[\[\]\',]','',x))
data_fi['colors']

0          W
1          W
2          W
3          W
4          W
        ... 
93638      U
93639      B
93640      R
93641      G
93642    R W
Name: colors, Length: 93643, dtype: object

In [20]:
data_fi.colors.unique()

array(['W', 'U', 'B', 'R', 'G', 'C', 'B G W', 'U W', 'B U', 'B U W',
       'B W', 'G R U', 'R U', 'B R', 'B G', 'G R', 'G R W', 'R W',
       'G U W', 'G U', 'B G R U W', 'G W', 'B R W', 'R U W', 'B G R',
       'B R U', 'B G U', 'B G U W', 'B R U W', 'G R U W', 'B G R W',
       'B G R U', 'W U', 'U B', 'U G', 'W R', 'U R', 'R G', 'W G', 'W B',
       'Green', 'Blue White', 'White', 'Black Green', 'U R G'],
      dtype=object)

In [21]:
color_map = {
    'White': 'W',   
    'Blue': 'U',    
    'Black': 'B',   
    'Red': 'R',     
    'Green': 'G',   
    'Colorless': 'C' 
}

In [22]:
data_fi.loc[:,'colors'] = data_fi['colors'].replace(color_map,regex=True)

In [23]:
data_fi.colors.unique()

array(['W', 'U', 'B', 'R', 'G', 'C', 'B G W', 'U W', 'B U', 'B U W',
       'B W', 'G R U', 'R U', 'B R', 'B G', 'G R', 'G R W', 'R W',
       'G U W', 'G U', 'B G R U W', 'G W', 'B R W', 'R U W', 'B G R',
       'B R U', 'B G U', 'B G U W', 'B R U W', 'G R U W', 'B G R W',
       'B G R U', 'W U', 'U B', 'U G', 'W R', 'U R', 'R G', 'W G', 'W B',
       'U R G'], dtype=object)

In [24]:
data_fi.loc[:,'colors'] = data_fi['colors'].apply(lambda x: " ".join(sorted(x.split())))

In [25]:
data_fi.colors.unique()

array(['W', 'U', 'B', 'R', 'G', 'C', 'B G W', 'U W', 'B U', 'B U W',
       'B W', 'G R U', 'R U', 'B R', 'B G', 'G R', 'G R W', 'R W',
       'G U W', 'G U', 'B G R U W', 'G W', 'B R W', 'R U W', 'B G R',
       'B R U', 'B G U', 'B G U W', 'B R U W', 'G R U W', 'B G R W',
       'B G R U'], dtype=object)

#### color_idetity

In [26]:
data_fi['color_identity'].isna().sum()

np.int64(10471)

In [27]:
data_fi['color_identity'].unique()

array(["['W']", "['U']", "['B']", "['R']", "['G']", nan,
       "['B', 'G', 'R', 'U', 'W']", "['U', 'W']", "['R', 'W']",
       "['G', 'W']", "['B', 'W']", "['G', 'R']", "['B', 'G']",
       "['R', 'U']", "['B', 'R']", "['B', 'U']", "['G', 'U']",
       "['B', 'R', 'W']", "['B', 'G', 'W']", "['B', 'U', 'W']",
       "['G', 'R', 'U']", "['G', 'R', 'W']", "['G', 'U', 'W']",
       "['R', 'U', 'W']", "['B', 'G', 'R']", "['B', 'R', 'U']",
       "['B', 'G', 'U']", "['B', 'G', 'U', 'W']", "['B', 'R', 'U', 'W']",
       "['G', 'R', 'U', 'W']", "['B', 'G', 'R', 'W']",
       "['B', 'G', 'R', 'U']"], dtype=object)

In [28]:
data_fi['color_identity']

0             ['W']
1             ['W']
2             ['W']
3             ['W']
4             ['W']
            ...    
93638         ['U']
93639         ['B']
93640         ['R']
93641         ['G']
93642    ['R', 'W']
Name: color_identity, Length: 93643, dtype: object

In [29]:
data_fi.loc[data_fi['color_identity'].isna(),'color_identity'] = 'C'

In [30]:
data_fi.loc[:,'color_identity'] = data_fi['color_identity'].apply(lambda x:re.sub(r'[\[\]\',]','',x))

In [31]:
data_fi['color_identity']

0          W
1          W
2          W
3          W
4          W
        ... 
93638      U
93639      B
93640      R
93641      G
93642    R W
Name: color_identity, Length: 93643, dtype: object

In [32]:
data_fi['color_identity'].unique()

array(['W', 'U', 'B', 'R', 'G', 'C', 'B G R U W', 'U W', 'R W', 'G W',
       'B W', 'G R', 'B G', 'R U', 'B R', 'B U', 'G U', 'B R W', 'B G W',
       'B U W', 'G R U', 'G R W', 'G U W', 'R U W', 'B G R', 'B R U',
       'B G U', 'B G U W', 'B R U W', 'G R U W', 'B G R W', 'B G R U'],
      dtype=object)

#### rarity

In [33]:
data_fi['rarity'].isna().sum()

np.int64(0)

In [34]:
data_fi['rarity'].unique()

array(['Uncommon', 'Common', 'Rare', 'Mythic', 'Special', 'Bonus'],
      dtype=object)

#### Artist

In [35]:
data_fi['artist'].isna().sum()

np.int64(9)

In [36]:
data_fi[data_fi['artist'].isna()]

,name,layout,mana_cost,cmc,colors,color_identity,type,supertypes,subtypes,rarity,artist,power,toughness,loyalty,printings,original_text,set_name,set
78624,Plains,normal,NaN,0.0,C,W,Basic Land — Plains,['Basic'],['Plains'],Rare,NaN,NaN,NaN,NaN,"['10E', '2ED', '2XM', '30A', '3ED', '40K', '4B...",NaN,Secret Lair Drop,SLD
78625,Island,normal,NaN,0.0,C,U,Basic Land — Island,['Basic'],['Island'],Rare,NaN,NaN,NaN,NaN,"['10E', '2ED', '2XM', '30A', '3ED', '40K', '4B...",NaN,Secret Lair Drop,SLD
78626,Swamp,normal,NaN,0.0,C,B,Basic Land — Swamp,['Basic'],['Swamp'],Rare,NaN,NaN,NaN,NaN,"['10E', '2ED', '2XM', '30A', '3ED', '40K', '4B...",NaN,Secret Lair Drop,SLD
78627,Mountain,normal,NaN,0.0,C,R,Basic Land — Mountain,['Basic'],['Mountain'],Rare,NaN,NaN,NaN,NaN,"['10E', '2ED', '2XM', '30A', '3ED', '40K', '4B...",NaN,Secret Lair Drop,SLD
78628,Forest,normal,NaN,0.0,C,G,Basic Land — Forest,['Basic'],['Forest'],Rare,NaN,NaN,NaN,NaN,"['10E', '2ED', '2XM', '30A', '3ED', '40K', '4B...",NaN,Secret Lair Drop,SLD
78955,Terramorphic Expanse,normal,NaN,0.0,C,C,Land,NaN,NaN,Rare,NaN,NaN,NaN,NaN,"['10E', '40K', 'AFC', 'ARC', 'BRC', 'C13', 'C1...",NaN,Secret Lair Drop,SLD
79120,Relentless Rats,normal,1BB,3.0,B,B,Creature — Rat,NaN,['Rat'],Rare,NaN,2,2,NaN,"['10E', '5DN', 'A25', 'M10', 'M11', 'PLST', 'P...",NaN,Secret Lair Drop,SLD
85611,"Look at Me, I'm R&D",normal,2W,3.0,W,W,Enchantment,NaN,NaN,Rare,NaN,NaN,NaN,NaN,"['UND', 'UNH']","As Look at Me, I'm R&D comes into play, choose...",Unsanctioned,UND
86363,"Look at Me, I'm R&D",normal,2W,3.0,W,W,Enchantment,NaN,NaN,Rare,NaN,NaN,NaN,NaN,"['UND', 'UNH']","As Look at Me, I'm R&D comes into play, choose...",Unhinged,UNH


In [37]:
data_fi.loc[data_fi['artist'].isna(),'artist'] = 'WOTC'

In [38]:
data_fi['artist'].isna().sum()

np.int64(0)

#### Power

In [39]:
data_fi['power'].isna().sum()

np.int64(49191)

In [40]:
data_fi['power'].unique()

array(['4', '3', nan, '0', '2', '1', '11', '5', '6', '7', '*', '8', '9',
       '15', '12', '10', '16', '13', '2+*', '-1', '1+*', '?', '18', '99',
       '∞', '+3', '+2', '+1', '1.5', '.5', '2.5', '3.5', '*²', '+4', '+0'],
      dtype=object)

In [41]:
data_fi[data_fi['toughness']=='7-']

,name,layout,mana_cost,cmc,colors,color_identity,type,supertypes,subtypes,rarity,artist,power,toughness,loyalty,printings,original_text,set_name,set


In [42]:
data_fi.loc[:,'power'] = data_fi['power'].str.replace('+','',regex=False)

In [43]:
data_fi['power'].unique()

array(['4', '3', nan, '0', '2', '1', '11', '5', '6', '7', '*', '8', '9',
       '15', '12', '10', '16', '13', '2*', '-1', '1*', '?', '18', '99',
       '∞', '1.5', '.5', '2.5', '3.5', '*²'], dtype=object)

In [44]:
data_fi.loc[:,'power'] = data_fi['power'].str.replace(r'^\*$', '0', regex=True)

In [45]:
data_fi['power'].unique()

array(['4', '3', nan, '0', '2', '1', '11', '5', '6', '7', '8', '9', '15',
       '12', '10', '16', '13', '2*', '-1', '1*', '?', '18', '99', '∞',
       '1.5', '.5', '2.5', '3.5', '*²'], dtype=object)

In [46]:
data_fi.loc[:,'power'] = data_fi['power'].str.replace('*²', '1', regex=False)

In [47]:
data_fi['power'].unique()

array(['4', '3', nan, '0', '2', '1', '11', '5', '6', '7', '8', '9', '15',
       '12', '10', '16', '13', '2*', '-1', '1*', '?', '18', '99', '∞',
       '1.5', '.5', '2.5', '3.5'], dtype=object)

In [48]:
data_fi.loc[:,'power'] = data_fi['power'].str.replace('∞', 'inf', regex=False)
data_fi.loc[:,'power'] = data_fi['power'].str.replace('?', '1', regex=False)
data_fi.loc[:,'power'] = data_fi['power'].str.replace('*', '', regex=False)

In [49]:
data_fi['power'].unique()

array(['4', '3', nan, '0', '2', '1', '11', '5', '6', '7', '8', '9', '15',
       '12', '10', '16', '13', '-1', '18', '99', 'inf', '1.5', '.5',
       '2.5', '3.5'], dtype=object)

In [50]:
data_fi.loc[:,'power'] = pd.to_numeric(data_fi['power'])

In [51]:
data_fi['power'].unique()

array([4.0, 3.0, nan, 0.0, 2.0, 1.0, 11.0, 5.0, 6.0, 7.0, 8.0, 9.0, 15.0,
       12.0, 10.0, 16.0, 13.0, -1.0, 18.0, 99.0, inf, 1.5, 0.5, 2.5, 3.5],
      dtype=object)

#### Toughness

In [52]:
data_fi['toughness'].isna().sum()

np.int64(49191)

In [53]:
data_fi['toughness'].unique()

array(['4', '3', nan, '2', '1', '5', '6', '0', '11', '7', '*', '8', '9',
       '15', '12', '10', '16', '13', '2+*', '7-*', '-1', '1+*', '?',
       '*+1', '20', '17', '14', '99', '+3', '+1', '-0', '3.5', '.5',
       '2.5', '1.5', '*²', '+2', '+4', '+0'], dtype=object)

In [54]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace('+','',regex=False)

In [55]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace(r'^\*$', '0', regex=True)

In [56]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace('*', '', regex=False)

In [57]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace('²', '1', regex=False)

In [58]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace('?', '1', regex=False)

In [59]:
data_fi.loc[:,'toughness'] = data_fi['toughness'].str.replace('7-', '7', regex=False)

In [60]:
data_fi['toughness'].unique()

array(['4', '3', nan, '2', '1', '5', '6', '0', '11', '7', '8', '9', '15',
       '12', '10', '16', '13', '-1', '20', '17', '14', '99', '-0', '3.5',
       '.5', '2.5', '1.5'], dtype=object)

In [61]:
data_fi.loc[:,'toughness'] = pd.to_numeric(data_fi['toughness'])

#### loyalty

In [62]:
data_fi['loyalty'].unique()

array([nan, '3', '7', '6', '4', '5', 'X', '2', '*', '0', '1d4+1', '20'],
      dtype=object)

In [63]:
data_fi.loc[:,'loyalty'] = data_fi['loyalty'].str.replace('1d4+1', '2', regex=False)
data_fi.loc[:,'loyalty'] = data_fi['loyalty'].str.replace(r'[*X]', '0', regex=True)

In [64]:
data_fi['loyalty'].unique()

array([nan, '3', '7', '6', '4', '5', '0', '2', '20'], dtype=object)

#### printings

In [65]:
data_fi['printings'].isna().sum()

np.int64(0)

In [66]:
data_fi['printings'].unique()

array(["['10E', 'JUD', 'UMA']",
       "['10E', '8ED', '9ED', 'DDC', 'DVD', 'IMA', 'INV', 'JMP', 'P02', 'PLST', 'PS11', 'PSAL', 'S99']",
       "['10E', '9ED', 'EXO', 'P02', 'POR', 'PS11', 'S00', 'S99', 'TPR']",
       ..., "['ZEN']", "['ZNC']", "['ZNR']"], shape=(9625,), dtype=object)

In [67]:
data_fi.loc[:,'printings'] = data_fi['printings'].str.replace(r'[\[\],\']','',regex=True)

In [68]:
data_fi['printings'].unique()

array(['10E JUD UMA',
       '10E 8ED 9ED DDC DVD IMA INV JMP P02 PLST PS11 PSAL S99',
       '10E 9ED EXO P02 POR PS11 S00 S99 TPR', ..., 'ZEN', 'ZNC', 'ZNR'],
      shape=(9625,), dtype=object)

#### Type

In [69]:
s = data_fi['type'].unique()
t = data_fi['type'].apply(lambda x: x.split("—")[0])
# pd.options.display.max_rows = len(s)
t.unique()

array(['Creature ', 'Sorcery', 'Enchantment', 'Instant',
       'Legendary Creature ', 'Enchantment ', 'Artifact',
       'Artifact Creature ', 'Legendary Artifact', 'Artifact ', 'Land',
       'Basic Land ', 'Land ', 'Tribal Enchantment ', 'Tribal Instant ',
       'Legendary Planeswalker ', 'Enchantment Creature ',
       'World Enchantment', 'Snow Artifact', 'Instant ',
       'Legendary Artifact Creature ', 'Legendary Artifact ',
       'Legendary Land', 'Legendary Snow Land', 'Artifact Land',
       'Legendary Enchantment Artifact', 'Legendary Enchantment',
       'Sorcery ', 'Legendary Enchantment Creature ',
       'Legendary Enchantment ', 'Legendary Sorcery', 'Snow Creature ',
       'Tribal Sorcery ', 'Legendary Land ', 'Snow Land ', 'Vanguard',
       'Instant Creature ', 'Summon ', 'Summon Wolf',
       'Elemental Instant ', 'Planeswalker ', 'Enchantment Land',
       'Land Creature ', 'Legendary Planeswalker', 'Basic Land',
       'Conspiracy', 'Snow Enchantment', 'Snow En

In [82]:
MAGIC_CARD_TYPES = [
    "CREATURE",      # Criatura
    "SORCERY",       # Feitiço
    "INSTANT",       # Mágica Instantânea
    "ENCHANTMENT",   # Encantamento
    "ARTIFACT",      # Artefato
    "PLANESWALKER",  # Planinalta
    "LAND",          # Terreno
    "TRIBAL",        # Tribal (tipo especial)
    # "SAGA",          # Saga (tipo especial)
    "BATTLE",         # Batalha (tipo especial)
    'CONSPIRACY',
    'SCHEME',
    'KINDRED',
    'SUMMON',
    'VANGUARD',
    'PLANE',
    'PHENOMENON',
    'HERO',
    'STICKERS',
    'UNIVERSEWALKER',
    '34'
    
    
]


In [83]:

types = data_fi['type']
types = types.apply(lambda x:x.replace("—","").split())


In [84]:
def crd_type(row):
    types = []
    for t in row:
        if t.upper() in MAGIC_CARD_TYPES:
            types.append(t.upper())
    if len(types) == 0:
        print(row)
    return ' '.join(types)

In [85]:
types = types.apply(crd_type)

In [86]:
types.value_counts()

type
CREATURE                40636
LAND                    10579
SORCERY                 10164
INSTANT                 10079
ENCHANTMENT              8807
ARTIFACT                 7462
ARTIFACT CREATURE        2945
PLANESWALKER             1523
ENCHANTMENT CREATURE      530
PLANE                     244
VANGUARD                  119
ARTIFACT LAND             117
SCHEME                     70
BATTLE                     52
TRIBAL INSTANT             51
STICKERS                   48
TRIBAL ENCHANTMENT         41
TRIBAL SORCERY             35
CONSPIRACY                 27
PHENOMENON                 25
ENCHANTMENT ARTIFACT       18
SUMMON                     17
HERO                       16
TRIBAL ARTIFACT             8
LAND CREATURE               7
ENCHANTMENT LAND            7
HERO ARTIFACT               5
INSTANT CREATURE            4
KINDRED ENCHANTMENT         2
ARTIFACT ENCHANTMENT        2
UNIVERSEWALKER              1
EATURECRAY                  1
CREATURE HERO               1
Name:

In [88]:
data_fi.loc[:,'type_cleanned'] = types

In [222]:
data_fi.columns

Index(['name', 'layout', 'mana_cost', 'cmc', 'colors', 'color_identity',
       'type', 'supertypes', 'subtypes', 'rarity', 'artist', 'power',
       'toughness', 'loyalty', 'printings', 'original_text', 'legalities',
       'set_name', 'type_cleanned'],
      dtype='object')

In [78]:
data_fi['type'].value_counts()

type
Sorcery                                  9823
Instant                                  9755
Artifact                                 5007
Enchantment                              4919
Land                                     4913
                                         ... 
Legendary Creature — Spirit Artificer       1
Creature — Elemental Spider                 1
Legendary Creature — Dinosaur Pirate        1
Creature — Goblin Rogue Wizard              1
Creature — Plant Rhino                      1
Name: count, Length: 3146, dtype: int64

In [89]:
data_fi.loc[:,'type_cleanned'] = data_fi['type_cleanned'].str.replace(r'SUMMON','CREATURE',regex=True)

In [90]:
data_fi['type'].value_counts()

type
Sorcery                                  9823
Instant                                  9755
Artifact                                 5007
Enchantment                              4919
Land                                     4913
                                         ... 
Legendary Creature — Spirit Artificer       1
Creature — Elemental Spider                 1
Legendary Creature — Dinosaur Pirate        1
Creature — Goblin Rogue Wizard              1
Creature — Plant Rhino                      1
Name: count, Length: 3146, dtype: int64

#### Subtypes

In [246]:
data_fi['subtypes'].isna().sum()

np.int64(36009)

In [233]:
data_fi[data_fi['subtypes'].isna()]['type_cleanned'].unique()

array(['SORCERY', 'ENCHANTMENT', 'INSTANT', 'ARTIFACT', 'LAND',
       'ARTIFACT LAND', 'ENCHANTMENT ARTIFACT', 'VANGUARD', 'CREATURE',
       'ENCHANTMENT LAND', 'PLANESWALKER', 'CONSPIRACY', 'PHENOMENON',
       'SCHEME', 'STICKERS', 'HERO'], dtype=object)

In [234]:
data_fi['subtypes'].unique()

array(["['Human', 'Cleric']", "['Angel']", nan, ...,
       "['Ogre', 'Shaman', 'Ally']", "['Elf', 'Scout', 'Ranger', 'Ally']",
       "['Cat', 'Horror']"], shape=(2376,), dtype=object)

In [235]:
data_fi.loc[:,'subtypes'] = data_fi['subtypes'].str.replace(r'[\[\],\']','',regex=True)

In [247]:
data_fi['subtypes'].unique()

array(['Human Cleric', 'Angel', nan, ..., 'Ogre Shaman Ally',
       'Elf Scout Ranger Ally', 'Cat Horror'], shape=(2385,), dtype=object)

In [237]:
no_subtype = data_fi[data_fi['subtypes'].isna()]

In [245]:
no_subtype[no_subtype['type_cleanned']=='CREATURE'][['name','type','type_cleanned','subtypes']]

,name,type,type_cleanned,subtypes
18934,Throat Wolf,Summon Wolf,CREATURE,NaN
19059,Throat Wolf,Summon Wolf,CREATURE,NaN
26435,Nameless Race,Creature,CREATURE,NaN
56498,Aswan Jaguar,Summon Jaguar,CREATURE,NaN
56500,Faerie Dragon,Summon Dragon,CREATURE,NaN
56501,Goblin Polka Band,Summon Goblin,CREATURE,NaN
56505,Prismatic Dragon,Summon Dragon,CREATURE,NaN
56506,Rainbow Knights,Summon Knights,CREATURE,NaN
57125,1996 World Champion,Summon Legend,CREATURE,NaN
57126,Shichifukujin Dragon,Summon Dragon,CREATURE,NaN


In [259]:
data_fi.loc[data_fi['type_cleanned']=='CREATURE','subtypes'] = data_fi.loc[data_fi['type_cleanned']=='CREATURE','subtypes'].fillna(data_fi['type'].str.split('Summon').str[-1])

In [262]:
data_fi[data_fi['name']=='Dr. Julius Jumblemorph']

,name,layout,mana_cost,cmc,colors,color_identity,type,supertypes,subtypes,rarity,artist,power,toughness,loyalty,printings,original_text,legalities,set_name,type_cleanned
85330,Dr. Julius Jumblemorph,normal,{2}{G}{W},4.0,"['G', 'W']","['G', 'W']",Legendary Creature,['Legendary'],NaN,Mythic,Simon Dominic,4,4,NaN,"['ULST', 'UST']",NaN,NaN,The List (Unfinity Foil Edition),CREATURE
86561,Dr. Julius Jumblemorph,normal,{2}{G}{W},4.0,"['G', 'W']","['G', 'W']",Legendary Creature,['Legendary'],NaN,Mythic,Simon Dominic,4,4,NaN,"['UPLIST', 'UST']",NaN,NaN,The List (Unfinity Foil Edition),CREATURE
87104,Dr. Julius Jumblemorph,normal,{2}{G}{W},4.0,"['G', 'W']","['G', 'W']",Legendary Creature,['Legendary'],NaN,Mythic,Simon Dominic,4,4,NaN,"['ULST', 'UST']",Dr. Julius Jumblemorph is every creature type ...,NaN,Unstable,CREATURE


In [261]:
data_fi.loc[data_fi['subtypes']=='Legendary Creature','subtypes'] = np.nan

In [263]:
data_fi.loc[:,'subtypes'] = data_fi['subtypes'].str.replace(r'[\[\],\']','',regex=True)

In [264]:
data_fi['subtypes']

0         Human Cleric
1         Human Cleric
2                Angel
3                Angel
4                  NaN
             ...      
93638              NaN
93639              NaN
93640              NaN
93641              NaN
93642    Human Warrior
Name: subtypes, Length: 93643, dtype: object

In [265]:
data_fi['subtypes'].unique()

array(['Human Cleric', 'Angel', nan, ..., 'Ogre Shaman Ally',
       'Elf Scout Ranger Ally', 'Cat Horror'], shape=(2384,), dtype=object)

#### Supertypes

In [266]:
data_fi['supertypes'].isna().sum()

np.int64(77065)

In [267]:
data_fi['supertypes'].unique()

array([nan, "['Legendary']", "['Basic']", "['World']", "['Snow']",
       "['Legendary', 'Snow']", "['Basic', 'Snow']", "['Ongoing']",
       "['Host']"], dtype=object)

In [268]:
data_fi.loc[:,'supertypes'] = data_fi['supertypes'].str.replace(r'[\[\],\']','',regex=True)

In [269]:
data_fi['supertypes'].unique()

array([nan, 'Legendary', 'Basic', 'World', 'Snow', 'Legendary Snow',
       'Basic Snow', 'Ongoing', 'Host'], dtype=object)

In [ ]:
#### 